# Training Data Explorer


In [1]:
%%html
<style>
  /* 1) The wrapper enforces the “height = X% of width” */
  .iframe‐wrapper {
    position: relative;
    width: 100%;           /* full-width */
    padding-bottom: 55%;   /* ← 55% of width = height */
    height: 0;             /* collapse the wrapper’s own height (padding-bottom will “create” the height) */
  }

  /* 2) The iframe is absolutely positioned to fill the wrapper */
  .iframe‐wrapper iframe {
    position: absolute;
    top: 0;
    left: 0;
    width: 100%;    /* fill 100% of wrapper’s width */
    height: 100%;   /* fill 100% of wrapper’s (calculated) height */
    border: 0;
  }
</style>

<div class="iframe‐wrapper">
  <iframe 
    src="_static/stations_combined.html"
    scrolling="no"
    allowfullscreen
  ></iframe>
</div>

In [ ]:

## How to Use the Map Interface

Follow these steps to interact with the map, expand the sidebar menu, and set which labels (e.g., air pollutant, monitoring station names) appear on the map.

---

### 1. Expand the Sidebar Menu

1. **Locate the Arrow Icon**  
   - In the top-left corner of the map, you’ll see a small arrow (`▶` or `◀`).  
   - If the menu is collapsed, this arrow points **right**; if it’s expanded, it points **left**.

2. **Click the Arrow**  
   - Click once to expand the sidebar.  
   - The menu will slide out, revealing filter options, layer controls, and labeling settings.

<aside>
💡 When the menu is expanded, the arrow flips direction (now pointing left). Clicking it again will collapse the menu.
</aside>

---

### 2. Navigate to Label Settings

1. **Find the “Labels” Section**  
   - In the expanded sidebar, scroll down until you see a section labeled **“Labels”** (or “Display Options” / “Layer Labels”).  
   - This section controls which textual information appears on top of the map markers or overlays.

2. **Open the Label Dropdown or Checklist**  
   - Click on the dropdown arrow (▾) or the “Labels” header to reveal all available label options.

---

### 3. Choose Which Labels to Show

In the “Labels” section, you will typically see checkboxes or toggles for each type of label. Common label types include:

- **Pollutant Type**  
  - Displays the abbreviation (e.g., “PM₂.₅”, “NO₂”, “O₃”) next to each monitoring station or on each pollution hotspot.

- **Monitoring Station Name**  
  - Shows the official name (e.g., “Downtown Station”, “Westside Monitoring”) for every station icon on the map.

---

### 4. Adjust Layer Visibility (Optional)

**Switch Between Pollutant Layers**  
   - At the top of the sidebar, there may be a **“Layers”** section where you can choose which pollutant layer (e.g., PM₂.₅, NO₂, O₃) is rendered.  
   - Select the layer you’re interested in, and the map will recolor or re‐symbolize accordingly.

---

### 5. Re‐collapse the Sidebar (When You’re Done)

- Click the same arrow icon in the top-left to collapse the menu.  
- This gives you a full‐screen view of the map without sidebar distractions.

---

## Example Walkthrough

1. **Initial State**  
   - By default, the map loads with the sidebar collapsed (arrow pointing right).

